In [1]:
import pandas as pd
from py2neo import Node, Graph, Relationship, NodeMatcher

In [2]:
path = r'D:/Invoice_data_Demo.xls'
invoice_data = pd.read_excel(path, header = 0)

In [3]:
invoice_data.shape

(35, 27)

In [4]:
invoice_data[:3]

,发票名称,机器编号,发票代码,发票号码,开票日期,校验码,购买方名称,购买方纳税人识别号,购买方地址、电话,购买方开户行及账号,...,通行日期起,通行日期止,金额,税率,税额,价税合计（大写）,价税合计（小写）,收款人,复核,开票人
0,山东增值税电子普通发票,499099649091,37001700112,476941,2018年03月23日,8385227000109856742,武汉市车城物流有限公司,914201007483062457,武汉经济技术开发区车城大道7号84289348,中国农业银行股份有限公司武汉开发区支行17-071201040004598,...,20180201,20180212,2617.05,不征税,＊＊＊,贰仟陆佰壹拾柒元零伍分,￥2617.05,束曼,束曼,王敏
1,湖北增值税电子普通发票,499099661823,42001700112,13208805,2018年03月23日,10350711227340909622,武汉市车城物流有限公司,914201007483062457,武汉经济技术开发区车城大道7号84289348,中国农业银行股份有限公司武汉开发区支行17-071201040004598,...,20180208,20180208,239.66,3%,7.19,贰佰肆拾陆元捌角伍分,￥246.85,戴宗鸿,戴宗鸿,胡朝霞
2,湖北增值税电子普通发票,499098921515,42001700112,27908343,2018年03月23日,8110641107709826910,武汉市车城物流有限公司,914201007483062457,武汉经济技术开发区车城大道7号84289348,中国农业银行股份有限公司武汉开发区支行17-071201040004598,...,20180208,20180208,97.33,3%,2.92,壹佰元零贰角伍分,￥100.25,刘为先,刘为先,柯璐


In [5]:
len(invoice_data)

35

In [6]:
# 把发票名称抽取出来
invoice_name_list = []
for i in range(0, len(invoice_data)):
    invoice_name_list.append(invoice_data['发票名称'][i])

In [7]:
len(invoice_name_list) 

35

In [8]:
invoice_name_list = list(set(invoice_name_list)) #去重
invoice_name_list # 第一类label的节点

['广东增值税电子普通发票', '江西增值税电子普通发票', '山东增值税电子普通发票', '湖北增值税电子普通发票', '湖南增值税电子普通发票']

In [9]:
invoice_data.columns

Index(['发票名称', '机器编号', '发票代码', '发票号码', '开票日期', '校验码', '购买方名称', '购买方纳税人识别号',
       '购买方地址、电话', '购买方开户行及账号', '销售方名称', '销售方纳税人识别号', '销售方地址、电话', '销售方开户行及账号',
       '项目名称', '车牌号', '类型', '通行日期起', '通行日期止', '金额', '税率', '税额', '价税合计（大写）',
       '价税合计（小写）', '收款人', '复核', '开票人'],
      dtype='object')

In [10]:
len(invoice_data.columns)

27

In [34]:
# 把除了发票代码这一列去掉，所有的都抽取了，一共抽取了35 * 26 = 910个数据
invoice_value_list = []
for i in range(0, len(invoice_data)):
    for j in range(1, len(invoice_data.columns)):
        invoice_value_list.append(invoice_data[invoice_data.columns[j]][i])

In [35]:
len(invoice_value_list)

910

In [36]:
invoice_value_list = list(set(invoice_value_list))# 

In [37]:
len(invoice_value_list)

474

In [38]:
# 转变为str类型
invoice_value_list = [str(i) for i in invoice_value_list]

In [39]:
invoice_value_list[:5]

['王国平', '13.66', '中行咸宁分行571657534679', '16.83', '18.51']

#### 抽取关系

In [41]:
invoice_data.columns[0]

'发票名称'

In [42]:
invoice_data[invoice_data.columns[0]]

0     山东增值税电子普通发票
1     湖北增值税电子普通发票
2     湖北增值税电子普通发票
3     湖北增值税电子普通发票
4     湖北增值税电子普通发票
5     湖北增值税电子普通发票
6     湖南增值税电子普通发票
7     湖南增值税电子普通发票
8     广东增值税电子普通发票
9     广东增值税电子普通发票
10    广东增值税电子普通发票
11    广东增值税电子普通发票
12    广东增值税电子普通发票
13    广东增值税电子普通发票
14    广东增值税电子普通发票
15    广东增值税电子普通发票
16    广东增值税电子普通发票
17    广东增值税电子普通发票
18    广东增值税电子普通发票
19    广东增值税电子普通发票
20    广东增值税电子普通发票
21    广东增值税电子普通发票
22    广东增值税电子普通发票
23    广东增值税电子普通发票
24    广东增值税电子普通发票
25    江西增值税电子普通发票
26    江西增值税电子普通发票
27    江西增值税电子普通发票
28    湖北增值税电子普通发票
29    湖北增值税电子普通发票
30    湖北增值税电子普通发票
31    湖北增值税电子普通发票
32    湖北增值税电子普通发票
33    广东增值税电子普通发票
34    广东增值税电子普通发票
Name: 发票名称, dtype: object

In [50]:
len(invoice_data.columns)

27

In [66]:
name1_list = []
name2_list = []
rel_list = []
for i in range(0, len(invoice_data)): # 35
    for j in range(1, len(invoice_data.columns)): # 1--26
        name1_list.append(invoice_data[invoice_data.columns[0]][i]) # 第一类label属性的节点
        rel_list.append(invoice_data.columns[j]) # 关系
        name2_list.append(invoice_data[invoice_data.columns[j]][i])

In [67]:
len(name1_list),len(rel_list),len(name2_list)

(910, 910, 910)

In [68]:
name2_list

[499099649091,
 37001700112,
 476941,
 '2018年03月23日',
 8385227000109856742,
 '武汉市车城物流有限公司',
 914201007483062457,
 '武汉经济技术开发区车城大道7号84289348',
 '中国农业银行股份有限公司武汉开发区支行17-071201040004598',
 '山东高速集团有限公司电子收费中心',
 '913701005716756695',
 '山东省济南市高新区奥体中路5006号山东高速科研楼西附楼二层南侧210室0531-8925182',
 '威海市商业银行济南舜耕支行09720201090000000389',
 '*经营租赁*代收通行费',
 '鄂AHG248',
 '货车',
 20180201,
 20180212,
 2617.05,
 '不征税',
 '＊＊＊',
 '贰仟陆佰壹拾柒元零伍分',
 '￥2617.05',
 '束曼',
 '束曼',
 '王敏',
 499099661823,
 42001700112,
 13208805,
 '2018年03月23日',
 10350711227340909622,
 '武汉市车城物流有限公司',
 914201007483062457,
 '武汉经济技术开发区车城大道7号84289348',
 '中国农业银行股份有限公司武汉开发区支行17-071201040004598',
 '湖北阿深南高速公路发展有限公司',
 '914200007570200387',
 '湖北省大冶市大棋大道6号0711-3706965',
 '工行黄石市杭州路支行1803015719200009617',
 '*经营租赁*通行费',
 '鄂AHG248',
 '货车',
 20180208,
 20180208,
 239.66,
 '3%',
 '7.19',
 '贰佰肆拾陆元捌角伍分',
 '￥246.85',
 '戴宗鸿',
 '戴宗鸿',
 '胡朝霞',
 499098921515,
 42001700112,
 27908343,
 '2018年03月23日',
 8110641107709826910,
 '武汉市车城物流有限公司',
 914201007483062457,
 '武汉经济技术开发区

In [71]:
name2_list = [str(i) for i in name2_list] # 里面有整数和浮点数，要变成str

In [72]:
tuple_total = list(zip(name1_list,rel_list,name2_list))

In [74]:
len(tuple_total)

910

## 把数据导入neo4j中

In [75]:
graph = Graph('你的neo4j的地址', auth = ('账号','密码'))
graph.delete_all() # 清除neo4j里面的所有数据

In [76]:
label_1 = '发票名称'
label_2 = '发票值'

In [77]:
len(invoice_value_list)

474

In [78]:
invoice_name_list

['广东增值税电子普通发票', '江西增值税电子普通发票', '山东增值税电子普通发票', '湖北增值税电子普通发票', '湖南增值税电子普通发票']

In [80]:
#把节点导入neo4j中
def create_node(invoice_name_list, invoice_value_list):
    for name in invoice_name_list:
        node_1 = Node(label_1, name = name)
        graph.create(node_1)
    for name in invoice_value_list:
        node_2 = Node(label_2, name = name)
        graph.create(node_2)
create_node(invoice_name_list, invoice_value_list)

In [81]:
len(tuple_total)

910

In [82]:
tuple_total[0]

('山东增值税电子普通发票', '机器编号', '499099649091')

In [83]:
tuple_total[0][2]

'499099649091'

In [84]:
matcher = NodeMatcher(graph)
matcher

In [85]:
# 导入关系
for i in range(0, len(tuple_total)):
    rel = Relationship(matcher.match(label_1, name = tuple_total[i][0]).first(),
                      tuple_total[i][1],
                      matcher.match(label_2, name = tuple_total[i][2]).first()
                      )
    graph.create(rel)